URL KAGGLE: https://www.kaggle.com/arturcgs/casos-uti-m-dulo-4/edit

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
!pip install openpyxl 

In [3]:
pip install scikit-learn -U

In [4]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np

In [5]:
#setando a seed

np.random.seed(18664)

In [6]:
dados = pd.read_excel("/kaggle/input/covid19/Kaggle_Sirio_Libanes_ICU_Prediction.xlsx")
dados.head()

In [7]:
dados.describe()

In [8]:
dados['PATIENT_VISIT_IDENTIFIER'].value_counts()

In [9]:
#Tirando as strings q n dá pra usar

dados_limpos = dados.dropna()

x_columns = dados.describe().columns

In [10]:
#Montando os dfs de entrada e saída
y = dados_limpos['ICU']
x = dados_limpos[x_columns].drop(['ICU'], axis=1)

In [11]:
#Criando e treinando o modelo
modelo = LogisticRegression(solver='liblinear')
modelo.fit(x, y)

In [12]:
#testando o número de acertos
sum(modelo.predict(x) == y)

In [13]:
#testando o número de erros 

sum(modelo.predict(x) != y)

In [14]:
#Vendo acurácia
accuracy_score(y, modelo.predict(x))

In [15]:
#Separando grupos de treino e teste

x_train, x_test, y_train, y_test = train_test_split(x, y, stratify = y)

print(len(x_train), len(y_train))
print(len(x_test), len(y_test))

In [16]:
#Fzndo um modelo separando grupos de treino e teste

modelo = LogisticRegression(solver='liblinear')
modelo.fit(x_train, y_train)
y_prediction = modelo.predict(x_test)
accuracy_score(y_test, y_prediction)

In [17]:
y_test.value_counts(normalize = True)

In [18]:
y_train.value_counts(normalize = True)

In [19]:
#Podemos ver q cada linha n é um paciente. Cada paciente tem 5 linhas

dados['PATIENT_VISIT_IDENTIFIER'].value_counts()

Vamos então agrupar esses pacientes. 

Além disso, tbm podemos preencher os dados NaN, com valores vizinhos (anterior ou posterior)


In [20]:
#Separando features

features_continuas_colunas = dados.iloc[:, 13:-2].columns
features_categoricas= dados.iloc[:, :13]
saida = dados.iloc[:, -2:]

In [21]:
#Agrupando os dados contínuos e preenchendo seus NaNs

features_continuas = dados.groupby("PATIENT_VISIT_IDENTIFIER", as_index=False)[features_continuas_colunas].fillna(method='bfill').fillna(method='ffill')

In [22]:
#Juntar tudo em uma tabela só

dados_limpos = pd.concat([features_categoricas, features_continuas, saida], ignore_index = True, axis = 1)
dados_limpos.columns = dados.columns
dados_limpos.head()

In [23]:
dados_limpos.describe()

In [24]:
#Tirando oq sobrou de NaN

dados_limpos = dados_limpos.dropna() 

Blz, a gte separou por grupo e preencheu os NaNs q davam, dps dropamos o resto de NaNs.

Agr, é o seguinte, se um paciente chegou e foi pra uti logo na primeira linha, ele n serve pro meu modelo. Então vamos tirar esses caras.

In [25]:
#Tirando esses caras

a_remover = dados_limpos.query("WINDOW == '0-2' and ICU == 1")["PATIENT_VISIT_IDENTIFIER"].values

dados_limpos = dados_limpos.query("PATIENT_VISIT_IDENTIFIER not in @a_remover")

dados_limpos.describe()

Agr, usar tds as linhas dos pacientes é meio difícil. Então, pra simplificar, vamos usar só a primeira linha

Pra isso, eu preciso ver, se um paciente esteve na UTI em qualquer momento, eu preciso mudar a UTI da primeira janela dele pra 1. Caso contrário, ele continuar como 0. Aí eu seleciono só a primeira linha de cada paciente

In [26]:
def prepare_window(rows):
    if np.any(rows["ICU"]) == 1:
        #rows.query("WINDOW == '0-2'")['ICU'] = 1 FARIA A MSM COISA, MAS DEMORA UM POUCO MAIS
        rows.loc[rows['WINDOW'] == "0-2", "ICU"] = 1
    return rows.loc[rows['WINDOW'] == "0-2"]

dados_limpos = dados_limpos.groupby('PATIENT_VISIT_IDENTIFIER').apply(prepare_window)
dados_limpos.head()

Agr a gte pode refazer nosso modelo com os dados_limpos

In [30]:
#setando a seed

np.random.seed(18664)

#Montando os dfs de entrada e saída
y = dados_limpos['ICU']
x = dados_limpos[x_columns].drop(['ICU'], axis=1)

#Separando grupos de treino e teste

x_train, x_test, y_train, y_test = train_test_split(x, y, stratify = y)

#Fazendo o modelo

modelo = LogisticRegression(solver='liblinear')
modelo.fit(x_train, y_train)
y_prediction = modelo.predict(x_test)
accuracy_score(y_test, y_prediction)